
**Imports**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from collections import defaultdict

from functions import (card_stats, possession_stats, calculate_main_stats,
                       update_match_stats, null_percentage)

In [8]:
%load_ext autoreload

%autoreload 2

In [2]:
reference_positions_df = pd.read_csv('data/PositionReference.csv')
conn = sqlite3.connect('data/football_database.sqlite')
cursor = conn.cursor()

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

**Having a Look At Tables, Columns and Missing Data**

In [4]:
q = "SELECT name FROM sqlite_master WHERE type='table';"

pd.read_sql(q, conn)

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [5]:
q = """
SELECT *
FROM Country"""
country_df = pd.read_sql(q, conn)
null_percentage(country_df)

id      0.0
name    0.0
dtype: float64

In [6]:
country_df.duplicated().sum()

0

In [7]:
q = """
SELECT *
FROM League"""
league_df = pd.read_sql(q, conn)
null_percentage(league_df)

id            0.0
country_id    0.0
name          0.0
dtype: float64

In [8]:
league_df.duplicated().sum()

0

In [13]:
q = """
SELECT *
FROM Match
"""
matches_df = pd.read_sql(q, conn)
null_percentage(matches_df)

id                   0.000000
country_id           0.000000
league_id            0.000000
season               0.000000
stage                0.000000
date                 0.000000
match_api_id         0.000000
home_team_api_id     0.000000
away_team_api_id     0.000000
home_team_goal       0.000000
away_team_goal       0.000000
home_player_X1       7.009508
home_player_X2       7.009508
home_player_X3       7.051850
home_player_X4       7.051850
home_player_X5       7.051850
home_player_X6       7.051850
home_player_X7       7.051850
home_player_X8       7.051850
home_player_X9       7.051850
home_player_X10      7.051850
home_player_X11      7.051850
away_player_X1       7.051850
away_player_X2       7.051850
away_player_X3       7.051850
away_player_X4       7.051850
away_player_X5       7.051850
away_player_X6       7.051850
away_player_X7       7.051850
away_player_X8       7.051850
away_player_X9       7.055699
away_player_X10      7.055699
away_player_X11      7.078794
home_playe

In [14]:
matches_df.duplicated().sum()

0

In [16]:
nulls_by_season = matches_df.groupby('season').apply(lambda x:
                                                     (x.isnull().mean() * 100))
nulls_by_season

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
season,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008/2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.886951,25.886951,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.307877,25.886951,25.886951,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.217679,26.307877,25.646422,25.826819,26.488274,25.947084,25.886951,26.518340,25.736621,26.849068,26.037282,26.819002,26.157547,25.796753,26.247745,26.007216,26.398076,25.796753,26.337943,25.646422,26.488274,26.157547,26.578473,26.277811,56.374023,56.374023,56.374023,56.374023,56.374023,56.374023,56.374023,56.374023,12.868310,12.868310,12.868310,12.958509,12.958509,12.958509,13.259170,13.259170,13.259170,12.988575,12.988575,12.988575,100.000000,100.000000,100.000000,13.168972,13.168972,13.168972,12.868310,12.868310,12.868310,12.958509,12.958509,12.958509,12.928443,12.928443,12.928443,12.958509,12.958509,12.958509
2009/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,7.430341,4.210526,5.448916,4.241486,4.613003,5.572755,4.984520,3.715170,4.179567,4.210526,5.356037,6.006192,4.086687,5.077399,4.117647,4.767802,5.696594,4.953560,3.777090,4.427245,4.953560,5.572755,6.315789,55.015480,55.015480,55.015480,55.015480,55.015480,55.015480,55.015480,55.015480,13.126935,13.126935,13.126935,13.405573,13.405573,13.405573,13.343653,13.343653,13.343653,13.157895,13.157895,13.157895,100.000000,100.000000,100.000000,13.312693,13.312693,13.312693,13.591331,13.591331,13.591331,13.312693,13.312693,13.312693,13.219814,13.219814,13.219814,13.188854,13.188854,13.188854
2010/2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.423313,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.361963,7.423313,3.006135,3.343

In [18]:
q = """
SELECT *
FROM Player
"""
player_df = pd.read_sql(q, conn)
null_percentage(player_df)

id                    0.0
player_api_id         0.0
player_name           0.0
player_fifa_api_id    0.0
birthday              0.0
height                0.0
weight                0.0
dtype: float64

In [20]:
player_df.duplicated().sum()

0

In [22]:
q = """
SELECT *
FROM Player_attributes
"""
player_attr_df = pd.read_sql(q, conn)
null_percentage(player_attr_df)

id                     0.000000
player_fifa_api_id     0.000000
player_api_id          0.000000
date                   0.000000
overall_rating         0.454402
potential              0.454402
preferred_foot         0.454402
attacking_work_rate    1.755645
defensive_work_rate    0.454402
crossing               0.454402
finishing              0.454402
heading_accuracy       0.454402
short_passing          0.454402
volleys                1.474633
dribbling              0.454402
curve                  1.474633
free_kick_accuracy     0.454402
long_passing           0.454402
ball_control           0.454402
acceleration           0.454402
sprint_speed           0.454402
agility                1.474633
reactions              0.454402
balance                1.474633
shot_power             0.454402
jumping                1.474633
stamina                0.454402
strength               0.454402
long_shots             0.454402
aggression             0.454402
interceptions          0.454402
position

In [24]:
player_attr_df.duplicated().sum()

0

In [26]:
q = """
SELECT *
FROM Team
"""
team_df = pd.read_sql(q, conn)
null_percentage(team_df)

id                  0.00000
team_api_id         0.00000
team_fifa_api_id    3.67893
team_long_name      0.00000
team_short_name     0.00000
dtype: float64

In [29]:
team_df.duplicated().sum()

0

In [31]:
q = """
SELECT *
FROM Team_attributes
"""
team_attr_df = pd.read_sql(q, conn)
null_percentage(team_attr_df)

id                                 0.000000
team_fifa_api_id                   0.000000
team_api_id                        0.000000
date                               0.000000
buildUpPlaySpeed                   0.000000
buildUpPlaySpeedClass              0.000000
buildUpPlayDribbling              66.460905
buildUpPlayDribblingClass          0.000000
buildUpPlayPassing                 0.000000
buildUpPlayPassingClass            0.000000
buildUpPlayPositioningClass        0.000000
chanceCreationPassing              0.000000
chanceCreationPassingClass         0.000000
chanceCreationCrossing             0.000000
chanceCreationCrossingClass        0.000000
chanceCreationShooting             0.000000
chanceCreationShootingClass        0.000000
chanceCreationPositioningClass     0.000000
defencePressure                    0.000000
defencePressureClass               0.000000
defenceAggression                  0.000000
defenceAggressionClass             0.000000
defenceTeamWidth                

In [32]:
team_attr_df.duplicated().sum()

0

- We have analyzed all the missing values for each table in our database. Its clear to see that in our Matches table, which contains information about each match, there is a high percentage of missing values in some of the columns. I investigated the missing values by season to see if we observe any trends. We observed that the earlier seasons tended to have a higher percentage of null values for the player positions and detailed match statistics. We also observed a moderate percentage of null values between the betting odds from different companies.

- We will be removing these null values when necessary during the analysis and handling them as we go through to understand the data more in depth.

**Formatting Player Positions**

In [33]:
q = """
SELECT *
FROM Match
"""
matches_df = pd.read_sql(q, conn)

In [34]:
q = """
SELECT *
FROM Player
"""
player_df = pd.read_sql(q, conn)

In [108]:
# Lets get all the player positions for each match in its own dataframe
# Get relevant columns for player_match df
player_match_cols = [
    'match_api_id', 'league_id', 'season', 'stage', 'team_api_id',
    'player_api_id', 'player_pos_x', 'player_pos_y'
]
# Create a blank df
players_position_df = pd.DataFrame()
# Converting data from wide -->  long format
for i in range(1, 12):
    # Select for player id and positions(x,y) for each home player
    home_cols = [f"home_player_{i}", f"home_player_X{i}", f"home_player_Y{i}"]
    # Add that to relevant columns
    home_info = matches_df[
        ['match_api_id', 'league_id', 'season', 'stage', 'home_team_api_id'] +
        home_cols]
    # Reset column names to match up
    home_info.columns = player_match_cols

    # Select for player id and positions(x,y) for each away player
    away_cols = [f"away_player_{i}", f"away_player_X{i}", f"away_player_Y{i}"]
    # Add that to relevant columns
    away_info = matches_df[
        ['match_api_id', 'league_id', 'season', 'stage', 'away_team_api_id'] +
        away_cols]
    # Reset column names
    away_info.columns = player_match_cols
    # concatenate both into one datframe
    players_position_df = pd.concat(
        [players_position_df, home_info, away_info], ignore_index=True)
# Remove null values (players)
player_positions_df = players_position_df.dropna(subset=['player_api_id'])

In [109]:
# Standardizing the x-coordinate for goalkeepers:
# Setting 'player_pos_x' to 5 where 'player_pos_y' is 1, which represents goalkeepers in the dataset
player_positions_df.loc[player_positions_df['player_pos_y'] == 1,
                        'player_pos_x'] = 5

In [110]:
# Get player positions by merging with ref csv
player_positions_df = pd.merge(player_positions_df,
                               reference_positions_df,
                               on=['player_pos_x', 'player_pos_y'],
                               how='left')
# Merge with player_df to get corresponding name
player_positions_df = pd.merge(player_positions_df,
                               player_df[['player_api_id', 'player_name']],
                               on=['player_api_id'],
                               how='left')

In [117]:
# Select relevant columns
player_positions_df_clean = player_positions_df[[
    'player_name', 'player_api_id', 'match_api_id', 'league_id', 'team_api_id',
    'season', 'role_xy'
]].rename(columns={'role_xy': 'position'})
player_positions_df_clean.head()

,player_name,player_api_id,match_api_id,league_id,team_api_id,season,position
0,Mark Volders,39890.0,493016,1,9996,2008/2009,GK
1,Wouter Biebauw,38327.0,493017,1,8203,2008/2009,GK
2,Bertrand Laquait,95597.0,493018,1,9986,2008/2009,GK
3,Boubacar Barry Copa,30934.0,493020,1,9994,2008/2009,GK
4,Stijn Stijnen,37990.0,493021,1,8342,2008/2009,GK


In [135]:
player_positions = pd.read_csv('player_positions.csv', index_col=0)

- The first observation i had was that we have all the player positions in wide format, that are represented by home_player_X1 until home_player_X11 . I decided that it would be easier to understand if we convert that into long format, allowing us to have the name of the player, and there corresponding position for each match row by row. 


- After extracting that information we saved the new dataframe to a csv file, allowing us to easily access it later in the analysis.

**Parsing XML**

In [122]:
# Finding all column names that contain xml in the matches table
xml_cols = [
    col for col in matches_df.columns
    if matches_df[col].astype(str).str.startswith('<').any()
]

In [123]:
# Grab only xml columns and drop null values
xml_parse_df = matches_df[xml_cols].dropna()
idxs = xml_parse_df.index

In [124]:
# # Cleaned dataframe
matches_clean = matches_df.iloc[idxs].copy()

In [125]:
# Stats we are intersted in
stats_to_track = [
    'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession'
]

In [126]:
for idx, row in matches_clean.iterrows():
    # Extract team ids from the current row
    home_team_id = row['home_team_api_id']
    away_team_id = row['away_team_api_id']
    # Loop through each stat we are interested in
    for stat in stats_to_track:
        # Extract the XML data for current stat
        xml_data = row[stat]
        # Call our functions to parse the xml into usable data
        home_stats, away_stats = update_match_stats(xml_data, home_team_id,
                                                    away_team_id, stat)
        # Update the dataframe with home team stats
        for e_type, value in home_stats.items():
            matches_clean.at[idx, f"{e_type}_home"] = value
        # Update the dataframe with away team stats
        for e_type, value in away_stats.items():
            matches_clean.at[idx, f'{e_type}_away'] = value

In [127]:
new_columns = [
    'shoton_home', 'shoton_away', 'shotoff_home', 'shotoff_away',
    'foulcommit_home', 'foulcommit_away', 'yellow_card_home', 'red_card_home',
    'cross_home', 'cross_away', 'corner_home', 'corner_away',
    'possession_home', 'possession_away', 'yellow_card_away', 'red_card_away'
]

In [128]:
# Replace the Nan with 0s
matches_clean[new_columns] = matches_clean[new_columns].fillna(0)
# Drop old xml columns
matches_clean = matches_clean.drop(columns=stats_to_track)
matches_clean = matches_clean.drop(columns=['goal'])

In [132]:
matches_clean.to_csv('match_detailed.csv')

- Our matches table had a few columns that contained XML data. We created a few functions to extract some of the important match statistics for each team (cards,shots on goal,etc). We then appended new columns that represented each statistic giving us more detailed overview of each match. Not all of the matches contained the relevant information, so we extracted what was available.


- We then proceeded to export the new dataframe into a csv file, once again this will allow us to access the data in our analysis notebook if needed.

**Extracting Player Ratings Per Match**

In [3]:
# Create Player id column names
home_ids = [f"home_player_{i}" for i in range(1, 12)]
away_ids = [f"away_player_{i}" for i in range(1, 12)]
player_columns = home_ids + away_ids

In [20]:
q = """
SELECT *
FROM Player_attributes
"""
df_playerattr = pd.read_sql(q, conn)
numeric_stats = df_playerattr.select_dtypes(include=np.number)
numeric_stats = numeric_stats.columns[3:-1]

In [19]:
# Extract relevant player stats
q = f"""
SELECT date,
       p.player_api_id,
       player_name,
       {','.join(numeric_stats)}
FROM player_attributes pa
JOIN player p ON pa.player_api_id = p.player_api_id
"""
player_ratings = pd.read_sql(q, conn)

In [12]:
# Drop null values
player_ratings = player_ratings.dropna(subset='overall_rating')

In [13]:
# Extract relevant match data
q = f"""
SELECT date ,
       match_api_id,
       home_team_api_id,
       away_team_api_id,
       home_team_goal,
       away_team_goal,
       CASE 
           WHEN home_team_goal > away_team_goal THEN 'home_win'
           WHEN away_team_goal > home_team_goal THEN 'away_win'
           ELSE 'draw'
           END as match_outcome,
       {', '.join(home_ids)},
       {', '.join(away_ids)}
       
FROM match
"""
matches = pd.read_sql(q, conn)

In [14]:
# Drop rows where any of the player columns have null values
matches = matches.dropna(subset=player_columns)

In [133]:
# Sort ratings to get the latest player ratings
player_ratings = player_ratings.sort_values('date', ascending=False)

# Create column names for home and away team statsz
home_columns = [f'home_{stat}' for stat in numeric_stats]
away_columns = [f'away_{stat}' for stat in numeric_stats]

# Loop over each type of team's player IDs and their corresponding columns
for ids, columns in zip([home_ids, away_ids], [home_columns, away_columns]):
    # Apply the 'get_most_recent_ratings' function to each match
    matches[columns] = matches.progress_apply(get_most_recent_ratings,
                                              axis=1,
                                              args=(player_ratings, ids, numeric_stats))

In [ ]:
matches.to_csv('matches_with_attributes.csv')

- The code blocks above utilized our get_most_recent_ratings function in order to compute and average the overall player ratings per team. This allowed to us to have extra columns for each match containing the average player ratings of each team combined, allowing us to get an overall rating for each team based on their players.

- We then exported to csv so we can use this later in our analysis/machine learning models.